In [50]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import statistics
import pickle
import csv
from nilearn import plotting
from scipy import stats
import nibabel as nib

In [51]:
def make_4d(dir_list, output_name):
    #Loop through each of the 3D scan paths in dir_list and make a 4D array out of them
    #First, get the shape of the 3D array
    img = nib.load(dir_list[0])
    data = img.get_fdata()
    shape = data.shape
    #Then, make a 4D array of zeros
    fourd_data = np.zeros((shape[0], shape[1], shape[2], len(dir_list)))
    #Then, loop through each 3D array and add it to the 4D array
    for i in range(len(dir_list)):
        print(dir_list[i])
        img = nib.load(dir_list[i])
        data = img.get_fdata()
        fourd_data[:,:,:,i] = data
    #Then save the 4D array as a nifti file
    affine = img.affine
    header = img.header
    new_img = nib.Nifti1Image(fourd_data, affine, header)
    nib.save(new_img, output_name)
    print("Saved 4D array as ", output_name)

In [52]:
#Converts CSVs to the mat format for FSL randomise.
def csv_to_mat(csv_path, out_path):
    
    with open(csv_path) as f:
        data = list(csv.reader(f))

    del data[0] #removes column titles
    num_row = str(len(data))
    num_col = str(len(data[0]))
    
    #write headers
    os.system("echo /NumWaves " +  num_col + " > " + out_path)
    os.system("echo /NumPoints " +  num_row + " >> " + out_path)
    os.system("echo /Matrix >> " + out_path)
    
    for line in data:
        line_txt = ' '.join(line)
        os.system("echo " +  line_txt + " >> " + out_path)
    print("Generated " + out_path)

# Delta

In [53]:
def get_delta_row(df, patno):
    mini_df = df[df['PATNO'] == patno]
    mini_df = mini_df.diff()
    mini_df = mini_df.dropna(how='all')
    mini_df['PATNO'] = patno
    return mini_df

In [54]:
#Read in the DF sorted by PATNO
corr_df = pd.read_pickle('table.pickle')
corr_df = corr_df[['PATNO', 'AGE_AT_VISIT', 'SEX', 'COHORT', 'NP3TOT', 'NHY', 'MCATOT','MMSETOT','GDSTOT','ABNORM','Depression']].copy()
corr_df = corr_df.loc[corr_df["PATNO"] != 3857] #Remove corrupted scans
corr_df = corr_df.loc[corr_df["PATNO"] != 3869]
corr_df = corr_df.loc[corr_df["COHORT"] != 2] #Remove Healthy Controls
corr_df = corr_df.drop('COHORT', axis=1)

patno_list_corr = list(corr_df.PATNO.unique()) #list of subjects
diff_df = pd.DataFrame(columns=list(corr_df)) #empty df with same columns

for i in patno_list_corr:
    diff_row = get_delta_row(corr_df, i)
    frames = [diff_df, diff_row]
    diff_df = pd.concat(frames, axis=0)

#Invert all of the columns that got diff'd in the wrong order
diff_df_positiveAge = diff_df[diff_df['AGE_AT_VISIT'] > 0]
diff_df_negativeAge = diff_df[diff_df['AGE_AT_VISIT'] < 0].mul(-1)
diff_df_negativeAge['PATNO'] = diff_df_negativeAge['PATNO'].apply(lambda x: x*-1)
diff_df_negativeAge['SEX'] = diff_df_negativeAge['SEX'].apply(lambda x: x*-1)
diff_df_negativeAge['ABNORM'] = diff_df_negativeAge['ABNORM'].apply(lambda x: x*-1)
frames = [diff_df_positiveAge, diff_df_negativeAge]
diff_df_final = pd.concat(frames, axis=0).sort_values('PATNO')

#Drop columns we don't need, or don't have complete data in.
diff_df_final = diff_df_final.drop('MCATOT', axis=1)
diff_df_final = diff_df_final.drop('MMSETOT', axis=1)
diff_df_final = diff_df_final.drop('ABNORM', axis=1)
diff_df_final = diff_df_final.drop('Depression', axis=1)
diff_df_final = diff_df_final.drop('PATNO', axis=1)
diff_df_final = diff_df_final.drop('SEX', axis=1)
diff_df_final = diff_df_final.replace(-0.0, 0.0)

diff_df_final['NP3TOT'] = diff_df_final['NP3TOT']/diff_df_final['AGE_AT_VISIT'] #avg change per year
diff_df_final['NHY'] = diff_df_final['NHY']/diff_df_final['AGE_AT_VISIT']
diff_df_final['GDSTOT'] = diff_df_final['GDSTOT']/diff_df_final['AGE_AT_VISIT']

diff_df_final = diff_df_final.round(1)
diff_df_final['INTERCEPT']=1.0

#display(diff_df_final)
diff_df_final.to_csv('/home/jacob/Desktop/analysis/correlation_delta_DM.csv', index=False) #Save the df as a CSV
csv_to_mat('/home/jacob/Desktop/analysis/correlation_delta_DM.csv', '/home/jacob/Desktop/analysis/correlation_delta_DM.mat') # Convert the CSV to mat

/tmp/ipykernel_3253/2905774919.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  diff_df = pd.concat(frames, axis=0)


Generated /home/jacob/Desktop/analysis/correlation_delta_DM.mat


In [55]:
corr_con = "/NumWaves 5\n/NumPoints 4\n/Matrix\n1 0 0 0 0\n0 1 0 0 0\n0 0 1 0 0\n0 0 0 1 0"
with open("/home/jacob/Desktop/analysis/correlation_delta_t.con", "w") as text_file:
    text_file.write(corr_con)

In [56]:
os.system("singularity run ../singufsl_img.simg randomise -i /home/jacob/Desktop/analysis/WR2GT_4D_PARKINSONS.nii.gz -o /home/jacob/Desktop/analysis/results1000/correlation_delta -d /home/jacob/Desktop/analysis/correlation_delta_DM.mat -t /home/jacob/Desktop/analysis/correlation_delta_t.con -n 1000 -m /home/jacob/Desktop/tessa-etal-2014_2/tessa-etal-2014/ANTs-derivatives/GT/GT_template0.nii.gz -T")

randomise options: -i /home/jacob/Desktop/analysis/WR2GT_4D_PARKINSONS.nii.gz -o /home/jacob/Desktop/analysis/results1000/correlation_delta -d /home/jacob/Desktop/analysis/correlation_delta_DM.mat -t /home/jacob/Desktop/analysis/correlation_delta_t.con -n 1000 -m /home/jacob/Desktop/tessa-etal-2014_2/tessa-etal-2014/ANTs-derivatives/GT/GT_template0.nii.gz -T 
Loading Data: 
Data loaded
2.19989e+12 permutations required for exhaustive test of t-test 1
Doing 1000 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Starting permutation 18
Starting permutation 19
Starting permutation 20
Starting permutation 21
S

Starting permutation 313
Starting permutation 314
Starting permutation 315
Starting permutation 316
Starting permutation 317
Starting permutation 318
Starting permutation 319
Starting permutation 320
Starting permutation 321
Starting permutation 322
Starting permutation 323
Starting permutation 324
Starting permutation 325
Starting permutation 326
Starting permutation 327
Starting permutation 328
Starting permutation 329
Starting permutation 330
Starting permutation 331
Starting permutation 332
Starting permutation 333
Starting permutation 334
Starting permutation 335
Starting permutation 336
Starting permutation 337
Starting permutation 338
Starting permutation 339
Starting permutation 340
Starting permutation 341
Starting permutation 342
Starting permutation 343
Starting permutation 344
Starting permutation 345
Starting permutation 346
Starting permutation 347
Starting permutation 348
Starting permutation 349
Starting permutation 350
Starting permutation 351
Starting permutation 352


Starting permutation 641
Starting permutation 642
Starting permutation 643
Starting permutation 644
Starting permutation 645
Starting permutation 646
Starting permutation 647
Starting permutation 648
Starting permutation 649
Starting permutation 650
Starting permutation 651
Starting permutation 652
Starting permutation 653
Starting permutation 654
Starting permutation 655
Starting permutation 656
Starting permutation 657
Starting permutation 658
Starting permutation 659
Starting permutation 660
Starting permutation 661
Starting permutation 662
Starting permutation 663
Starting permutation 664
Starting permutation 665
Starting permutation 666
Starting permutation 667
Starting permutation 668
Starting permutation 669
Starting permutation 670
Starting permutation 671
Starting permutation 672
Starting permutation 673
Starting permutation 674
Starting permutation 675
Starting permutation 676
Starting permutation 677
Starting permutation 678
Starting permutation 679
Starting permutation 680


Starting permutation 969
Starting permutation 970
Starting permutation 971
Starting permutation 972
Starting permutation 973
Starting permutation 974
Starting permutation 975
Starting permutation 976
Starting permutation 977
Starting permutation 978
Starting permutation 979
Starting permutation 980
Starting permutation 981
Starting permutation 982
Starting permutation 983
Starting permutation 984
Starting permutation 985
Starting permutation 986
Starting permutation 987
Starting permutation 988
Starting permutation 989
Starting permutation 990
Starting permutation 991
Starting permutation 992
Starting permutation 993
Starting permutation 994
Starting permutation 995
Starting permutation 996
Starting permutation 997
Starting permutation 998
Starting permutation 999
Starting permutation 1000
Critical Value for: /home/jacob/Desktop/analysis/results1000/correlation_delta_tfce_tstat1 is: 15306
2.55455e+19 permutations required for exhaustive test of t-test 2
Doing 1000 random permutations
S

Starting permutation 293
Starting permutation 294
Starting permutation 295
Starting permutation 296
Starting permutation 297
Starting permutation 298
Starting permutation 299
Starting permutation 300
Starting permutation 301
Starting permutation 302
Starting permutation 303
Starting permutation 304
Starting permutation 305
Starting permutation 306
Starting permutation 307
Starting permutation 308
Starting permutation 309
Starting permutation 310
Starting permutation 311
Starting permutation 312
Starting permutation 313
Starting permutation 314
Starting permutation 315
Starting permutation 316
Starting permutation 317
Starting permutation 318
Starting permutation 319
Starting permutation 320
Starting permutation 321
Starting permutation 322
Starting permutation 323
Starting permutation 324
Starting permutation 325
Starting permutation 326
Starting permutation 327
Starting permutation 328
Starting permutation 329
Starting permutation 330
Starting permutation 331
Starting permutation 332


...

Starting permutation 929
Starting permutation 930
Starting permutation 931
Starting permutation 932
Starting permutation 933
Starting permutation 934
Starting permutation 935
Starting permutation 936
Starting permutation 937
Starting permutation 938
Starting permutation 939
Starting permutation 940
Starting permutation 941
Starting permutation 942
Starting permutation 943
Starting permutation 944
Starting permutation 945
Starting permutation 946
Starting permutation 947
Starting permutation 948
Starting permutation 949
Starting permutation 950
Starting permutation 951
Starting permutation 952
Starting permutation 953
Starting permutation 954
Starting permutation 955
Starting permutation 956
Starting permutation 957
Starting permutation 958
Starting permutation 959
Starting permutation 960
Starting permutation 961
Starting permutation 962
Starting permutation 963
Starting permutation 964
Starting permutation 965
Starting permutation 966
Starting permutation 967
Starting permutation 968


Starting permutation 253
Starting permutation 254
Starting permutation 255
Starting permutation 256
Starting permutation 257
Starting permutation 258
Starting permutation 259
Starting permutation 260
Starting permutation 261
Starting permutation 262
Starting permutation 263
Starting permutation 264
Starting permutation 265
Starting permutation 266
Starting permutation 267
Starting permutation 268
Starting permutation 269
Starting permutation 270
Starting permutation 271
Starting permutation 272
Starting permutation 273
Starting permutation 274
Starting permutation 275
Starting permutation 276
Starting permutation 277
Starting permutation 278
Starting permutation 279
Starting permutation 280
Starting permutation 281
Starting permutation 282
Starting permutation 283
Starting permutation 284
Starting permutation 285
Starting permutation 286
Starting permutation 287
Starting permutation 288
Starting permutation 289
Starting permutation 290
Starting permutation 291
Starting permutation 292


Starting permutation 581
Starting permutation 582
Starting permutation 583
Starting permutation 584
Starting permutation 585
Starting permutation 586
Starting permutation 587
Starting permutation 588
Starting permutation 589
Starting permutation 590
Starting permutation 591
Starting permutation 592
Starting permutation 593
Starting permutation 594
Starting permutation 595
Starting permutation 596
Starting permutation 597
Starting permutation 598
Starting permutation 599
Starting permutation 600
Starting permutation 601
Starting permutation 602
Starting permutation 603
Starting permutation 604
Starting permutation 605
Starting permutation 606
Starting permutation 607
Starting permutation 608
Starting permutation 609
Starting permutation 610
Starting permutation 611
Starting permutation 612
Starting permutation 613
Starting permutation 614
Starting permutation 615
Starting permutation 616
Starting permutation 617
Starting permutation 618
Starting permutation 619
Starting permutation 620


Starting permutation 909
Starting permutation 910
Starting permutation 911
Starting permutation 912
Starting permutation 913
Starting permutation 914
Starting permutation 915
Starting permutation 916
Starting permutation 917
Starting permutation 918
Starting permutation 919
Starting permutation 920
Starting permutation 921
Starting permutation 922
Starting permutation 923
Starting permutation 924
Starting permutation 925
Starting permutation 926
Starting permutation 927
Starting permutation 928
Starting permutation 929
Starting permutation 930
Starting permutation 931
Starting permutation 932
Starting permutation 933
Starting permutation 934
Starting permutation 935
Starting permutation 936
Starting permutation 937
Starting permutation 938
Starting permutation 939
Starting permutation 940
Starting permutation 941
Starting permutation 942
Starting permutation 943
Starting permutation 944
Starting permutation 945
Starting permutation 946
Starting permutation 947
Starting permutation 948


0

# Baseline

In [57]:
#Read in the DF sorted by PATNO
corr_df = pd.read_pickle('table.pickle')
corr_df = corr_df[['PATNO', 'EVENT_ID', 'AGE_AT_VISIT', 'SEX', 'COHORT', 'NP3TOT', 'NHY', 'MCATOT','MMSETOT','GDSTOT','ABNORM','Depression']].copy()
corr_df = corr_df.loc[corr_df["PATNO"] != 3857] #Remove corrupted scans
corr_df = corr_df.loc[corr_df["PATNO"] != 3869]
corr_df = corr_df.loc[corr_df["COHORT"] != 2] #Remove Healthy Controls
corr_df = corr_df.loc[corr_df.EVENT_ID == "BL"] #Only keep baseline 
corr_df = corr_df.drop('COHORT', axis=1)
corr_df = corr_df.drop('EVENT_ID', axis=1)
patno_list_corr = list(corr_df.PATNO.unique()) #list of subjects

#Drop columns we don't need, or don't have complete data in.
corr_df = corr_df.drop('MCATOT', axis=1)
corr_df = corr_df.drop('MMSETOT', axis=1)
corr_df = corr_df.drop('ABNORM', axis=1)
corr_df = corr_df.drop('Depression', axis=1)
corr_df = corr_df.drop('PATNO', axis=1)
corr_df['INTERCEPT']=1.0

display(corr_df)
corr_df.to_csv('/home/jacob/Desktop/analysis/correlation_BL_DM.csv', index=False) #Save the df as a CSV
csv_to_mat('/home/jacob/Desktop/analysis/correlation_BL_DM.csv', '/home/jacob/Desktop/analysis/correlation_BL_DM.mat') # Convert the CSV to mat

,AGE_AT_VISIT,SEX,NP3TOT,NHY,GDSTOT,INTERCEPT
1619,69.3,1.0,13.0,1.0,6.0,1.0
1655,57.2,1.0,16.0,1.0,6.0,1.0
1736,49.1,0.0,12.0,1.0,6.0,1.0
1753,60.0,0.0,16.0,2.0,4.0,1.0
2253,51.4,1.0,26.0,2.0,6.0,1.0
6927,65.5,1.0,24.0,2.0,1.0,1.0
7320,49.2,1.0,26.0,2.0,4.0,1.0
8935,52.0,0.0,24.0,1.0,3.0,1.0
9064,48.5,1.0,7.0,1.0,5.0,1.0
9846,75.5,1.0,27.0,1.0,2.0,1.0


Generated /home/jacob/Desktop/analysis/correlation_BL_DM.mat


In [58]:
corr_con = "/NumWaves 6\n/NumPoints 3\n/Matrix\n0 0 1 0 0 0\n0 0 0 1 0 0\n0 0 0 0 1 0"
with open("/home/jacob/Desktop/analysis/correlation_BL_t.con", "w") as text_file:
    text_file.write(corr_con)

In [59]:
os.system("singularity run ../singufsl_img.simg randomise -i /home/jacob/Desktop/analysis/WR2GT_4D_PARKINSONS.nii.gz -o /home/jacob/Desktop/analysis/results1000/correlation_BL -d /home/jacob/Desktop/analysis/correlation_BL_DM.mat -t /home/jacob/Desktop/analysis/correlation_BL_t.con -n 1000 -m /home/jacob/Desktop/tessa-etal-2014_2/tessa-etal-2014/ANTs-derivatives/GT/GT_template0.nii.gz -T")

randomise options: -i /home/jacob/Desktop/analysis/WR2GT_4D_PARKINSONS.nii.gz -o /home/jacob/Desktop/analysis/results1000/correlation_BL -d /home/jacob/Desktop/analysis/correlation_BL_DM.mat -t /home/jacob/Desktop/analysis/correlation_BL_t.con -n 1000 -m /home/jacob/Desktop/tessa-etal-2014_2/tessa-etal-2014/ANTs-derivatives/GT/GT_template0.nii.gz -T 
Loading Data: 
Data loaded
1.77399e+17 permutations required for exhaustive test of t-test 1
Doing 1000 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Starting permutation 18
Starting permutation 19
Starting permutation 20
Starting permutation 21
Starting p

Starting permutation 314
Starting permutation 315
Starting permutation 316
Starting permutation 317
Starting permutation 318
Starting permutation 319
Starting permutation 320
Starting permutation 321
Starting permutation 322
Starting permutation 323
Starting permutation 324
Starting permutation 325
Starting permutation 326
Starting permutation 327
Starting permutation 328
Starting permutation 329
Starting permutation 330
Starting permutation 331
Starting permutation 332
Starting permutation 333
Starting permutation 334
Starting permutation 335
Starting permutation 336
Starting permutation 337
Starting permutation 338
Starting permutation 339
Starting permutation 340
Starting permutation 341
Starting permutation 342
Starting permutation 343
Starting permutation 344
Starting permutation 345
Starting permutation 346
Starting permutation 347
Starting permutation 348
Starting permutation 349
Starting permutation 350
Starting permutation 351
Starting permutation 352
Starting permutation 353


Starting permutation 642
Starting permutation 643
Starting permutation 644
Starting permutation 645
Starting permutation 646
Starting permutation 647
Starting permutation 648
Starting permutation 649
Starting permutation 650
Starting permutation 651
Starting permutation 652
Starting permutation 653
Starting permutation 654
Starting permutation 655
Starting permutation 656
Starting permutation 657
Starting permutation 658
Starting permutation 659
Starting permutation 660
Starting permutation 661
Starting permutation 662
Starting permutation 663
Starting permutation 664
Starting permutation 665
Starting permutation 666
Starting permutation 667
Starting permutation 668
Starting permutation 669
Starting permutation 670
Starting permutation 671
Starting permutation 672
Starting permutation 673
Starting permutation 674
Starting permutation 675
Starting permutation 676
Starting permutation 677
Starting permutation 678
Starting permutation 679
Starting permutation 680
Starting permutation 681


Starting permutation 970
Starting permutation 971
Starting permutation 972
Starting permutation 973
Starting permutation 974
Starting permutation 975
Starting permutation 976
Starting permutation 977
Starting permutation 978
Starting permutation 979
Starting permutation 980
Starting permutation 981
Starting permutation 982
Starting permutation 983
Starting permutation 984
Starting permutation 985
Starting permutation 986
Starting permutation 987
Starting permutation 988
Starting permutation 989
Starting permutation 990
Starting permutation 991
Starting permutation 992
Starting permutation 993
Starting permutation 994
Starting permutation 995
Starting permutation 996
Starting permutation 997
Starting permutation 998
Starting permutation 999
Starting permutation 1000
Critical Value for: /home/jacob/Desktop/analysis/results1000/correlation_BL_tfce_tstat1 is: 17730.3
3.87988e+06 permutations required for exhaustive test of t-test 2
Doing 1000 random permutations
Starting permutation 1 (Unp

Starting permutation 294
Starting permutation 295
Starting permutation 296
Starting permutation 297
Starting permutation 298
Starting permutation 299
Starting permutation 300
Starting permutation 301
Starting permutation 302
Starting permutation 303
Starting permutation 304
Starting permutation 305
Starting permutation 306
Starting permutation 307
Starting permutation 308
Starting permutation 309
Starting permutation 310
Starting permutation 311
Starting permutation 312
Starting permutation 313
Starting permutation 314
Starting permutation 315
Starting permutation 316
Starting permutation 317
Starting permutation 318
Starting permutation 319
Starting permutation 320
Starting permutation 321
Starting permutation 322
Starting permutation 323
Starting permutation 324
Starting permutation 325
Starting permutation 326
Starting permutation 327
Starting permutation 328
Starting permutation 329
Starting permutation 330
Starting permutation 331
Starting permutation 332
Starting permutation 333


...

Starting permutation 950
Starting permutation 951
Starting permutation 952
Starting permutation 953
Starting permutation 954
Starting permutation 955
Starting permutation 956
Starting permutation 957
Starting permutation 958
Starting permutation 959
Starting permutation 960
Starting permutation 961
Starting permutation 962
Starting permutation 963
Starting permutation 964
Starting permutation 965
Starting permutation 966
Starting permutation 967
Starting permutation 968
Starting permutation 969
Starting permutation 970
Starting permutation 971
Starting permutation 972
Starting permutation 973
Starting permutation 974
Starting permutation 975
Starting permutation 976
Starting permutation 977
Starting permutation 978
Starting permutation 979
Starting permutation 980
Starting permutation 981
Starting permutation 982
Starting permutation 983
Starting permutation 984
Starting permutation 985
Starting permutation 986
Starting permutation 987
Starting permutation 988
Starting permutation 989


Starting permutation 274
Starting permutation 275
Starting permutation 276
Starting permutation 277
Starting permutation 278
Starting permutation 279
Starting permutation 280
Starting permutation 281
Starting permutation 282
Starting permutation 283
Starting permutation 284
Starting permutation 285
Starting permutation 286
Starting permutation 287
Starting permutation 288
Starting permutation 289
Starting permutation 290
Starting permutation 291
Starting permutation 292
Starting permutation 293
Starting permutation 294
Starting permutation 295
Starting permutation 296
Starting permutation 297
Starting permutation 298
Starting permutation 299
Starting permutation 300
Starting permutation 301
Starting permutation 302
Starting permutation 303
Starting permutation 304
Starting permutation 305
Starting permutation 306
Starting permutation 307
Starting permutation 308
Starting permutation 309
Starting permutation 310
Starting permutation 311
Starting permutation 312
Starting permutation 313


Starting permutation 602
Starting permutation 603
Starting permutation 604
Starting permutation 605
Starting permutation 606
Starting permutation 607
Starting permutation 608
Starting permutation 609
Starting permutation 610
Starting permutation 611
Starting permutation 612
Starting permutation 613
Starting permutation 614
Starting permutation 615
Starting permutation 616
Starting permutation 617
Starting permutation 618
Starting permutation 619
Starting permutation 620
Starting permutation 621
Starting permutation 622
Starting permutation 623
Starting permutation 624
Starting permutation 625
Starting permutation 626
Starting permutation 627
Starting permutation 628
Starting permutation 629
Starting permutation 630
Starting permutation 631
Starting permutation 632
Starting permutation 633
Starting permutation 634
Starting permutation 635
Starting permutation 636
Starting permutation 637
Starting permutation 638
Starting permutation 639
Starting permutation 640
Starting permutation 641


Starting permutation 930
Starting permutation 931
Starting permutation 932
Starting permutation 933
Starting permutation 934
Starting permutation 935
Starting permutation 936
Starting permutation 937
Starting permutation 938
Starting permutation 939
Starting permutation 940
Starting permutation 941
Starting permutation 942
Starting permutation 943
Starting permutation 944
Starting permutation 945
Starting permutation 946
Starting permutation 947
Starting permutation 948
Starting permutation 949
Starting permutation 950
Starting permutation 951
Starting permutation 952
Starting permutation 953
Starting permutation 954
Starting permutation 955
Starting permutation 956
Starting permutation 957
Starting permutation 958
Starting permutation 959
Starting permutation 960
Starting permutation 961
Starting permutation 962
Starting permutation 963
Starting permutation 964
Starting permutation 965
Starting permutation 966
Starting permutation 967
Starting permutation 968
Starting permutation 969


0

# Follow-up

In [60]:
#Read in the DF sorted by PATNO
corr_df = pd.read_pickle('table.pickle')
corr_df = corr_df[['PATNO', 'EVENT_ID', 'AGE_AT_VISIT', 'SEX', 'COHORT', 'NP3TOT', 'NHY', 'MCATOT','MMSETOT','GDSTOT','ABNORM','Depression']].copy()
corr_df = corr_df.loc[corr_df["PATNO"] != 3857] #Remove corrupted scans
corr_df = corr_df.loc[corr_df["PATNO"] != 3869]
corr_df = corr_df.loc[corr_df["COHORT"] != 2] #Remove Healthy Controls
corr_df = corr_df.loc[corr_df.EVENT_ID != "BL"] #Only keep follow-ups 
corr_df = corr_df.drop('COHORT', axis=1)
corr_df = corr_df.drop('EVENT_ID', axis=1)
patno_list_corr = list(corr_df.PATNO.unique()) #list of subjects

#Drop columns we don't need, or don't have complete data in.
corr_df = corr_df.drop('MCATOT', axis=1)
corr_df = corr_df.drop('MMSETOT', axis=1)
corr_df = corr_df.drop('ABNORM', axis=1)
corr_df = corr_df.drop('Depression', axis=1)
corr_df = corr_df.drop('PATNO', axis=1)
corr_df['INTERCEPT']=1.0

display(corr_df)
corr_df.to_csv('/home/jacob/Desktop/analysis/correlation_FU_DM.csv', index=False) #Save the df as a CSV
csv_to_mat('/home/jacob/Desktop/analysis/correlation_FU_DM.csv', '/home/jacob/Desktop/analysis/correlation_FU_DM.mat') # Convert the CSV to mat

,AGE_AT_VISIT,SEX,NP3TOT,NHY,GDSTOT,INTERCEPT
1630,71.3,1.0,24.0,2.0,6.0,1.0
1668,61.2,1.0,18.0,2.0,7.0,1.0
1745,53.5,0.0,23.0,2.0,5.0,1.0
1765,64.1,0.0,37.0,2.0,4.0,1.0
2261,53.4,1.0,47.0,2.0,6.0,1.0
6935,67.5,1.0,21.0,2.0,5.0,1.0
7329,51.3,1.0,30.0,2.0,3.0,1.0
8946,55.9,0.0,36.0,2.0,3.0,1.0
9077,52.5,1.0,13.0,1.0,6.0,1.0
9854,77.5,1.0,25.0,2.0,3.0,1.0


Generated /home/jacob/Desktop/analysis/correlation_FU_DM.mat


In [61]:
corr_con = "/NumWaves 6\n/NumPoints 3\n/Matrix\n0 0 1 0 0 0\n0 0 0 1 0 0\n0 0 0 0 1 0"
with open("/home/jacob/Desktop/analysis/correlation_FU_t.con", "w") as text_file:
    text_file.write(corr_con)

In [62]:
os.system("singularity run ../singufsl_img.simg randomise -i /home/jacob/Desktop/analysis/WR2GT_4D_PARKINSONS.nii.gz -o /home/jacob/Desktop/analysis/results1000/correlation_fu -d /home/jacob/Desktop/analysis/correlation_FU_DM.mat -t /home/jacob/Desktop/analysis/correlation_FU_t.con -n 1000 -m /home/jacob/Desktop/tessa-etal-2014_2/tessa-etal-2014/ANTs-derivatives/GT/GT_template0.nii.gz -T")

randomise options: -i /home/jacob/Desktop/analysis/WR2GT_4D_PARKINSONS.nii.gz -o /home/jacob/Desktop/analysis/results1000/correlation_fu -d /home/jacob/Desktop/analysis/correlation_FU_DM.mat -t /home/jacob/Desktop/analysis/correlation_FU_t.con -n 1000 -m /home/jacob/Desktop/tessa-etal-2014_2/tessa-etal-2014/ANTs-derivatives/GT/GT_template0.nii.gz -T 
Loading Data: 
Data loaded
3.19318e+18 permutations required for exhaustive test of t-test 1
Doing 1000 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Starting permutation 18
Starting permutation 19
Starting permutation 20
Starting permutation 21
Starting p

Starting permutation 314
Starting permutation 315
Starting permutation 316
Starting permutation 317
Starting permutation 318
Starting permutation 319
Starting permutation 320
Starting permutation 321
Starting permutation 322
Starting permutation 323
Starting permutation 324
Starting permutation 325
Starting permutation 326
Starting permutation 327
Starting permutation 328
Starting permutation 329
Starting permutation 330
Starting permutation 331
Starting permutation 332
Starting permutation 333
Starting permutation 334
Starting permutation 335
Starting permutation 336
Starting permutation 337
Starting permutation 338
Starting permutation 339
Starting permutation 340
Starting permutation 341
Starting permutation 342
Starting permutation 343
Starting permutation 344
Starting permutation 345
Starting permutation 346
Starting permutation 347
Starting permutation 348
Starting permutation 349
Starting permutation 350
Starting permutation 351
Starting permutation 352
Starting permutation 353


Starting permutation 642
Starting permutation 643
Starting permutation 644
Starting permutation 645
Starting permutation 646
Starting permutation 647
Starting permutation 648
Starting permutation 649
Starting permutation 650
Starting permutation 651
Starting permutation 652
Starting permutation 653
Starting permutation 654
Starting permutation 655
Starting permutation 656
Starting permutation 657
Starting permutation 658
Starting permutation 659
Starting permutation 660
Starting permutation 661
Starting permutation 662
Starting permutation 663
Starting permutation 664
Starting permutation 665
Starting permutation 666
Starting permutation 667
Starting permutation 668
Starting permutation 669
Starting permutation 670
Starting permutation 671
Starting permutation 672
Starting permutation 673
Starting permutation 674
Starting permutation 675
Starting permutation 676
Starting permutation 677
Starting permutation 678
Starting permutation 679
Starting permutation 680
Starting permutation 681


Starting permutation 970
Starting permutation 971
Starting permutation 972
Starting permutation 973
Starting permutation 974
Starting permutation 975
Starting permutation 976
Starting permutation 977
Starting permutation 978
Starting permutation 979
Starting permutation 980
Starting permutation 981
Starting permutation 982
Starting permutation 983
Starting permutation 984
Starting permutation 985
Starting permutation 986
Starting permutation 987
Starting permutation 988
Starting permutation 989
Starting permutation 990
Starting permutation 991
Starting permutation 992
Starting permutation 993
Starting permutation 994
Starting permutation 995
Starting permutation 996
Starting permutation 997
Starting permutation 998
Starting permutation 999
Starting permutation 1000
Critical Value for: /home/jacob/Desktop/analysis/results1000/correlation_fu_tfce_tstat1 is: 18893
1330 permutations required for exhaustive test of t-test 2
Doing 1000 random permutations
Starting permutation 1 (Unpermuted d

Starting permutation 294
Starting permutation 295
Starting permutation 296
Starting permutation 297
Starting permutation 298
Starting permutation 299
Starting permutation 300
Starting permutation 301
Starting permutation 302
Starting permutation 303
Starting permutation 304
Starting permutation 305
Starting permutation 306
Starting permutation 307
Starting permutation 308
Starting permutation 309
Starting permutation 310
Starting permutation 311
Starting permutation 312
Starting permutation 313
Starting permutation 314
Starting permutation 315
Starting permutation 316
Starting permutation 317
Starting permutation 318
Starting permutation 319
Starting permutation 320
Starting permutation 321
Starting permutation 322
Starting permutation 323
Starting permutation 324
Starting permutation 325
Starting permutation 326
Starting permutation 327
Starting permutation 328
Starting permutation 329
Starting permutation 330
Starting permutation 331
Starting permutation 332
Starting permutation 333


...

Starting permutation 950
Starting permutation 951
Starting permutation 952
Starting permutation 953
Starting permutation 954
Starting permutation 955
Starting permutation 956
Starting permutation 957
Starting permutation 958
Starting permutation 959
Starting permutation 960
Starting permutation 961
Starting permutation 962
Starting permutation 963
Starting permutation 964
Starting permutation 965
Starting permutation 966
Starting permutation 967
Starting permutation 968
Starting permutation 969
Starting permutation 970
Starting permutation 971
Starting permutation 972
Starting permutation 973
Starting permutation 974
Starting permutation 975
Starting permutation 976
Starting permutation 977
Starting permutation 978
Starting permutation 979
Starting permutation 980
Starting permutation 981
Starting permutation 982
Starting permutation 983
Starting permutation 984
Starting permutation 985
Starting permutation 986
Starting permutation 987
Starting permutation 988
Starting permutation 989


Starting permutation 274
Starting permutation 275
Starting permutation 276
Starting permutation 277
Starting permutation 278
Starting permutation 279
Starting permutation 280
Starting permutation 281
Starting permutation 282
Starting permutation 283
Starting permutation 284
Starting permutation 285
Starting permutation 286
Starting permutation 287
Starting permutation 288
Starting permutation 289
Starting permutation 290
Starting permutation 291
Starting permutation 292
Starting permutation 293
Starting permutation 294
Starting permutation 295
Starting permutation 296
Starting permutation 297
Starting permutation 298
Starting permutation 299
Starting permutation 300
Starting permutation 301
Starting permutation 302
Starting permutation 303
Starting permutation 304
Starting permutation 305
Starting permutation 306
Starting permutation 307
Starting permutation 308
Starting permutation 309
Starting permutation 310
Starting permutation 311
Starting permutation 312
Starting permutation 313


Starting permutation 602
Starting permutation 603
Starting permutation 604
Starting permutation 605
Starting permutation 606
Starting permutation 607
Starting permutation 608
Starting permutation 609
Starting permutation 610
Starting permutation 611
Starting permutation 612
Starting permutation 613
Starting permutation 614
Starting permutation 615
Starting permutation 616
Starting permutation 617
Starting permutation 618
Starting permutation 619
Starting permutation 620
Starting permutation 621
Starting permutation 622
Starting permutation 623
Starting permutation 624
Starting permutation 625
Starting permutation 626
Starting permutation 627
Starting permutation 628
Starting permutation 629
Starting permutation 630
Starting permutation 631
Starting permutation 632
Starting permutation 633
Starting permutation 634
Starting permutation 635
Starting permutation 636
Starting permutation 637
Starting permutation 638
Starting permutation 639
Starting permutation 640
Starting permutation 641


Starting permutation 930
Starting permutation 931
Starting permutation 932
Starting permutation 933
Starting permutation 934
Starting permutation 935
Starting permutation 936
Starting permutation 937
Starting permutation 938
Starting permutation 939
Starting permutation 940
Starting permutation 941
Starting permutation 942
Starting permutation 943
Starting permutation 944
Starting permutation 945
Starting permutation 946
Starting permutation 947
Starting permutation 948
Starting permutation 949
Starting permutation 950
Starting permutation 951
Starting permutation 952
Starting permutation 953
Starting permutation 954
Starting permutation 955
Starting permutation 956
Starting permutation 957
Starting permutation 958
Starting permutation 959
Starting permutation 960
Starting permutation 961
Starting permutation 962
Starting permutation 963
Starting permutation 964
Starting permutation 965
Starting permutation 966
Starting permutation 967
Starting permutation 968
Starting permutation 969


0